In [1]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [2]:
from fastai.vision.all import *
import params

import wandb

In [3]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [4]:
path = Path(untar_data(URL, force_download=True))

In [5]:
path.ls()

(#3) [Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/labels')]

In [7]:
# What's the structure of the dataset we downloaded?
(path/'images').ls()

(#1000) [Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/43d4a202-ca890610.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/851ec515-15faca1d.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/6b73ccdd-00000000.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/912e44a6-dc2d2079.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/c7f68540-18a5e077.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/587e93da-9d2eef22.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/67a9f729-e9eb734e.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/489a53ad-e275e01c.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/9a35e283-7c9c7404.jpg'),Path('/home/iowomugisha/.fastai/data/bdd_simple_1k/images/21154886-00000000.jpg')...]

In [6]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)

    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                image,
                masks={
                    "predictions": {
                        "mask_data": mask_data,
                        "class_labels": class_labels,
                    }
                }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )

    return table

In [17]:
!wandb login

wandb: Currently logged in as: archimedesisaac. Use `wandb login --relogin` to force relogin


In [20]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

wandb: ERROR Error while calling W&B API: project not found (<Response [404]>)


Problem at: /home/iowomugisha/Projects/wandb-cicd/venv/lib/python3.11/site-packages/wandb/sdk/wandb_init.py 836 getcaller


CommError: Run initialization has timed out after 60.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-